## Modeling Finacial Fraud Transactions

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
model_df = pd.read_csv("/Users/sa17/Desktop/financial-fraud/financial-fraud-analysis/data/processed/cleanlog.csv")

model_df

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,170136.00,160296.36,0.00,0.00,0,0,0,1,0
1,1864.28,21249.00,19384.72,0.00,0.00,0,0,0,1,0
2,181.00,181.00,0.00,0.00,0.00,1,0,0,0,1
3,181.00,181.00,0.00,21182.00,0.00,1,1,0,0,0
4,11668.14,41554.00,29885.86,0.00,0.00,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
6264735,339682.13,339682.13,0.00,0.00,339682.13,1,1,0,0,0
6264736,6311409.28,6311409.28,0.00,0.00,0.00,1,0,0,0,1
6264737,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,1,0,0,0
6264738,850002.52,850002.52,0.00,0.00,0.00,1,0,0,0,1


In [4]:
# Declare feature vector and target variable 
X = model_df.drop(["isFraud"], axis=1)

y = model_df["isFraud"]

In [15]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [16]:
# check the shape of X_train and X_test
X_train.shape, X_test.shape

((5011792, 9), (1252948, 9))

In [ ]:
# Train Logistic Regression Model
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)

# Prediction Model
y_pred = lr.predict(X_test)


LOGISIC REGRESSION 0.5

ROC CURVE TO FIND BEST LINE 

KNN